In [1]:
import fiona
import pandas as pd
import geopandas as gpd
import csv

In [2]:
from collections import OrderedDict
    
layers = fiona.listlayers('ipbes_pollination_summary_hg_2018-08-16_12_21_-0700_036a5f15f926.gpkg')

In [3]:
layers

['grid_1_degree',
 'pollination_table_full',
 'pollination_table',
 'total_ppl_poll',
 'pollination_table_with_negative_ones',
 'region_NCPsummary_positive',
 'region_NCPsummary_negative',
 'production_table',
 'region_PCsummary_positive',
 'region_PCsummary_negative',
 'c_poll_for_violins',
 'd_poll_and_tot_for_violins',
 'relevant_population',
 'c_unrealized',
 'UNxNCP',
 'UNxNCP_ssp1_incrpot',
 'UNxNCP_ssp3_incrpot',
 'UNxNCP_ssp5_incrpot',
 'UNxNCP_ssp1_incrpot_or',
 'UNxNCP_ssp3_incrpot_or',
 'UNxNCP_ssp5_incrpot_or',
 'region_summary_capped',
 'region_summary',
 'region_popimpacted_ssp1',
 'region_popimpacted_ssp3',
 'region_popimpacted_ssp5',
 'region_popveryimpacted_ssp1',
 'region_popveryimpacted_ssp3',
 'region_popveryimpacted_ssp5']

In [4]:
all_datasets = dict()

for layer in layers:
    all_datasets[layer.lower()] = gpd.read_file(
        "ipbes_pollination_summary_hg_2018-08-16_12_21_-0700_036a5f15f926.gpkg", layer=layer
    )

In [5]:
all_datasets['relevant_population']

,fid,GRIDCODE,region,country,prod_total_realized_en_1d_cur,prod_total_realized_fo_1d_cur,prod_total_realized_va_1d_cur,prod_poll_dep_realized_en_1d_cur,prod_poll_dep_realized_fo_1d_cur,prod_poll_dep_realized_va_1d_cur,...,prod_poll_indep_va_ssp5,relevant_pop_cur,relevant_pop_ssp1,relevant_pop_ssp3,relevant_pop_ssp5,relevant_min_pop_cur,relevant_min_pop_ssp1,relevant_min_pop_ssp3,relevant_min_pop_ssp5,geometry
0,8845,8846,Eurasia,Finland,1.000202e+09,2.795330e+07,6.607354e+06,3.792434e+06,0.000000e+00,0.000000e+00,...,6.607354e+06,125575,152706,123749,173010,125575,152706,123749,173010,None
1,9204,9205,Eurasia,Finland,4.901934e+10,1.391889e+09,6.610467e+09,2.226972e+08,4.965414e+06,3.820599e+09,...,2.789868e+09,69314,78508,61138,91201,69314,78508,61138,91201,None
2,9205,9206,Eurasia,Finland,6.426054e+11,1.838986e+10,7.296157e+10,1.992002e+09,4.155600e+07,3.213111e+10,...,4.030735e+10,0,0,0,0,0,0,0,0,None
3,9206,9207,Eurasia,Finland,1.503560e+11,4.092868e+09,1.202020e+10,5.909089e+08,8.950065e+06,6.665564e+09,...,5.263059e+09,27423,29558,0,34888,0,0,0,0,None
4,9207,9208,Eurasia,Finland,4.891865e+08,1.273620e+07,3.831052e+06,1.624868e+06,3.845838e+02,1.922919e+02,...,3.830859e+06,45897,50656,39685,58809,45897,50656,39685,58809,None
5,9208,9209,Eurasia,Finland,3.452708e+08,9.105704e+06,2.713110e+06,9.596202e+05,0.000000e+00,0.000000e+00,...,2.713110e+06,17736,19138,14904,22516,17736,19138,14904,22516,None
6,9561,9562,Eurasia,Finland,1.207103e+11,3.411382e+09,9.856563e+09,4.340336e+08,6.740814e+06,5.141855e+09,...,4.699048e+09,128884,150292,120648,171320,0,150292,0,171320,None
7,9562,9563,Eurasia,Finland,8.554422e+11,2.435323e+10,8.312252e+10,2.831128e+09,5.132791e+07,3.932303e+10,...,4.346743e+10,0,0,0,0,0,0,0,0,None
8,9563,9564,Eurasia,Finland,5.535218e+11,1.549736e+10,6.036060e+10,2.135897e+09,4.080309e+07,3.120016e+10,...,2.883884e+10,0,0,0,0,0,0,0,0,None
9,9564,9565,Eurasia,Finland,4.050725e+11,1.140294e+10,3.659389e+10,1.292475e+09,2.226322e+07,1.687542e+10,...,1.958457e+10,0,0,0,0,0,0,0,0,None


### Fields we want to visualize:
NCP_cur,NCP_sspX, UN_cur, UN_sspX, or any of the changes... cNCP_, cUN_
### Preparing the data

en = energy , fo = Folate , va = vitamineA <br>
curr = current<br>
ssp1 = First scenario<br> <br> 
UN = Unmet need (current, ssp1 , ssp3 , ssp5)<br>
NCP = Nature's Contribution to Pollination(current, ssp1 , ssp3 , ssp5)<br>


In [6]:
unxncp = all_datasets['unxncp']

current = unxncp[['GRIDCODE','prod_poll_dep_unrealized_en_1d_cur','prod_poll_dep_unrealized_fo_1d_cur','prod_poll_dep_unrealized_va_1d_cur',
                  'nat_cont_poll_en_1d_cur','nat_cont_poll_fo_1d_cur','nat_cont_poll_va_1d_cur','prod_poll_dep_realized_en_1d_cur','prod_poll_dep_realized_fo_1d_cur'
                  ,'prod_poll_dep_realized_va_1d_cur','UN_cur','NCP_cur']]

ssp1 = unxncp[['GRIDCODE','prod_poll_dep_unrealized_en_1d_ssp1','prod_poll_dep_unrealized_fo_1d_ssp1','prod_poll_dep_unrealized_va_1d_ssp1',
                  'nat_cont_poll_en_1d_ssp1','nat_cont_poll_fo_1d_ssp1','nat_cont_poll_va_1d_ssp1','prod_poll_dep_realized_en_1d_ssp1','prod_poll_dep_realized_fo_1d_ssp1'
                  ,'prod_poll_dep_realized_va_1d_ssp1','UN_ssp1','NCP_ssp1']]

ssp3 = unxncp[['GRIDCODE','prod_poll_dep_unrealized_en_1d_ssp3','prod_poll_dep_unrealized_fo_1d_ssp3','prod_poll_dep_unrealized_va_1d_ssp3',
                  'nat_cont_poll_en_1d_ssp3','nat_cont_poll_fo_1d_ssp3','nat_cont_poll_va_1d_ssp3','prod_poll_dep_realized_en_1d_ssp3','prod_poll_dep_realized_fo_1d_ssp3'
                  ,'prod_poll_dep_realized_va_1d_ssp3','UN_ssp3','NCP_ssp3']]

ssp5 = unxncp[['GRIDCODE','prod_poll_dep_unrealized_en_1d_ssp5','prod_poll_dep_unrealized_fo_1d_ssp5','prod_poll_dep_unrealized_va_1d_ssp5',
                  'nat_cont_poll_en_1d_ssp5','nat_cont_poll_fo_1d_ssp5','nat_cont_poll_va_1d_ssp5','prod_poll_dep_realized_en_1d_ssp5','prod_poll_dep_realized_fo_1d_ssp5'
                  ,'prod_poll_dep_realized_va_1d_ssp5','UN_ssp5','NCP_ssp5']]


# Interesting Data 
NCP = unxncp[['GRIDCODE','NCP_cur','NCP_ssp1','NCP_ssp3','NCP_ssp5']]
UN = unxncp[['GRIDCODE','UN_cur','UN_ssp1','UN_ssp3','UN_ssp5']]

# Changes

cNCP = unxncp[['GRIDCODE','cNCP_ssp1','cNCP_ssp3','cNCP_ssp5']]
cUN = unxncp[['GRIDCODE','cUN_ssp1','cUN_ssp3','cUN_ssp5']]



# # Data to csv
# NCP.to_csv('NCP', sep='\t')


In [7]:
all_datasets['pollination_table_full']

,fid,geom,GRIDCODE,country,region,PCTU5,UW,prod_poll_dep_realized_en_1d_ssp5,nat_cont_poll_en_1d_ssp5,prod_poll_dep_unrealized_en_1d_ssp5,...,gpw_v4_e_atot_pop_30s_ssp3,gpw_v4_e_atot_pop_30s_ssp5,isag,c_poll_cont_nut_req_avg_1d_ssp1,c_poll_cont_nut_req_avg_1d_ssp3,c_poll_cont_nut_req_avg_1d_ssp5,d_poll_cont_nut_req_avg_1d_ssp1,d_poll_cont_nut_req_avg_1d_ssp3,d_poll_cont_nut_req_avg_1d_ssp5,geometry
0,8845,GP,8846,Finland,Eurasia,5.7,-999.0,3.492576e+06,0.920932,2.998582e+05,...,123749.796875,173010.359375,1,-0.182394,0.011380,-0.331555,-0.000002,1.438966e-07,-0.000004,None
1,9204,GP,9205,Finland,Eurasia,5.7,-999.0,2.081489e+08,0.821218,4.531472e+07,...,61138.570312,91201.968750,1,-0.121255,0.128436,-0.288955,-0.011524,1.220602e-02,-0.027461,None
2,9205,GP,9206,Finland,Eurasia,5.7,-999.0,1.755138e+09,0.523101,1.600121e+09,...,32481.710938,49574.121094,1,-0.009892,0.008775,-0.021472,-0.003538,3.138304e-03,-0.007679,None
3,9206,GP,9207,Finland,Eurasia,5.7,-999.0,4.447526e+08,0.701118,1.895948e+08,...,22747.699219,34888.214844,1,-0.040155,0.001025,-0.269395,-0.013775,3.516674e-04,-0.092417,None
4,9207,GP,9208,Finland,Eurasia,5.7,-999.0,1.596864e+06,0.982765,2.800423e+04,...,39685.875000,58809.265625,1,-0.108285,0.139090,-0.232960,-0.000002,2.028642e-06,-0.000003,None
5,9208,GP,9209,Finland,Eurasia,5.7,-999.0,9.325091e+05,0.961348,3.749262e+04,...,14904.937500,22516.632812,1,-0.073294,0.190029,-0.234527,-0.000002,4.218549e-06,-0.000005,None
6,9561,GP,9562,Finland,Eurasia,5.7,-999.0,3.533810e+08,0.618096,2.183440e+08,...,120648.953125,171320.562500,1,-0.186799,0.040157,-0.407653,-0.012853,2.763145e-03,-0.028050,None
7,9562,GP,9563,Finland,Eurasia,5.7,-999.0,2.121756e+09,0.494743,2.166849e+09,...,84828.453125,125197.062500,1,-0.008907,0.000216,-0.017317,-0.003092,7.507205e-05,-0.006011,None
8,9563,GP,9564,Finland,Eurasia,5.7,-999.0,1.684325e+09,0.625858,1.006902e+09,...,58049.367188,88441.234375,1,-0.007057,0.003700,-0.016032,-0.002453,1.286209e-03,-0.005573,None
9,9564,GP,9565,Finland,Eurasia,5.7,-999.0,1.058967e+09,0.538912,9.060419e+08,...,26568.490234,40648.304688,1,-0.007245,0.006934,-0.018516,-0.002548,2.438664e-03,-0.006512,None


**Description of the data** <br>

X1 = prod_poll_dep_unrealized (The unrealized production that is pollination dependant)<br>
X2 = prod_poll_dep_realized ( The realized production that is pollination dependant) <br>
Y = prod_poll_indep (production that is pollination independant)<br>
Z = nut_req ( nutrition required for the area)<br>

**Data Available for these timeframes:** 1850 - 1900 - 1910 - 1945 - 1980 - current - ssp1 - ssp3 - ssp5

In [8]:
pollination_table = all_datasets['pollination_table_full']

#  1850
# Z is not available for this year
data_1850 = pollination_table[['GRIDCODE','country',
                               'prod_poll_dep_unrealized_en_1d_1850','prod_poll_dep_unrealized_va_1d_1850','prod_poll_dep_unrealized_fo_1d_1850',
                               'prod_poll_dep_realized_en_1d_1850','prod_poll_dep_realized_va_1d_1850','prod_poll_dep_realized_fo_1d_1850',
                               'prod_total_realized_en_1d_1850','prod_total_realized_va_1d_1850','prod_total_realized_fo_1d_1850']]
data_1850.columns = ['GRIDCODE','name','X1_en','X1_va','X1_fo','X2_en','X2_va','X2_fo','X2+Y_en','X2+Y_va','X2+Y_fo']

#  1900
# Z is not available for this year
data_1900 = pollination_table[['GRIDCODE','country',
                               'prod_poll_dep_unrealized_en_1d_1900','prod_poll_dep_unrealized_va_1d_1900','prod_poll_dep_unrealized_fo_1d_1900',
                               'prod_poll_dep_realized_en_1d_1900','prod_poll_dep_realized_va_1d_1900','prod_poll_dep_realized_fo_1d_1900',
                               'prod_total_realized_en_1d_1900','prod_total_realized_va_1d_1900','prod_total_realized_fo_1d_1900']]
data_1900.columns = ['GRIDCODE','name','X1_en','X1_va','X1_fo','X2_en','X2_va','X2_fo','X2+Y_en','X2+Y_va','X2+Y_fo']

#  1910
# Z is not available for this year
data_1910 = pollination_table[['GRIDCODE','country',
                               'prod_poll_dep_unrealized_en_1d_1910','prod_poll_dep_unrealized_va_1d_1910','prod_poll_dep_unrealized_fo_1d_1910',
                               'prod_poll_dep_realized_en_1d_1910','prod_poll_dep_realized_va_1d_1910','prod_poll_dep_realized_fo_1d_1910',
                               'prod_total_realized_en_1d_1910','prod_total_realized_va_1d_1910','prod_total_realized_fo_1d_1910']]
data_1910.columns = ['GRIDCODE','name','X1_en','X1_va','X1_fo','X2_en','X2_va','X2_fo','X2+Y_en','X2+Y_va','X2+Y_fo']

#  1945
# Z is not available for this year
data_1945 = pollination_table[['GRIDCODE','country',
                               'prod_poll_dep_unrealized_en_1d_1945','prod_poll_dep_unrealized_va_1d_1945','prod_poll_dep_unrealized_fo_1d_1945',
                               'prod_poll_dep_realized_en_1d_1945','prod_poll_dep_realized_va_1d_1945','prod_poll_dep_realized_fo_1d_1945',
                               'prod_total_realized_en_1d_1945','prod_total_realized_va_1d_1945','prod_total_realized_fo_1d_1945']]
data_1945.columns = ['GRIDCODE','name','X1_en','X1_va','X1_fo','X2_en','X2_va','X2_fo','X2+Y_en','X2+Y_va','X2+Y_fo']

#  1980
# Z is not available for this year
data_1980 = pollination_table[['GRIDCODE','country',
                               'prod_poll_dep_unrealized_en_1d_1980','prod_poll_dep_unrealized_va_1d_1980','prod_poll_dep_unrealized_fo_1d_1980',
                               'prod_poll_dep_realized_en_1d_1980','prod_poll_dep_realized_va_1d_1980','prod_poll_dep_realized_fo_1d_1980',
                               'prod_total_realized_en_1d_1980','prod_total_realized_va_1d_1980','prod_total_realized_fo_1d_1980']]
data_1980.columns = ['GRIDCODE','name','X1_en','X1_va','X1_fo','X2_en','X2_va','X2_fo','X2+Y_en','X2+Y_va','X2+Y_fo']
data_1980

#  cur
data_cur = pollination_table[['GRIDCODE','country',
                               'prod_poll_dep_unrealized_en_1d_cur','prod_poll_dep_unrealized_va_1d_cur','prod_poll_dep_unrealized_fo_1d_cur',
                               'prod_poll_dep_realized_en_1d_cur','prod_poll_dep_realized_va_1d_cur','prod_poll_dep_realized_fo_1d_cur',
                               'prod_total_realized_en_1d_cur','prod_total_realized_va_1d_cur','prod_total_realized_fo_1d_cur',
                               'nut_req_en_1d_cur','nut_req_va_1d_cur','nut_req_fo_1d_cur']]

data_cur.columns = ['GRIDCODE','name','X1_en','X1_va','X1_fo','X2_en','X2_va','X2_fo','X2+Y_en','X2+Y_va','X2+Y_fo','Z_en','Z_va','Z_fo']

#  ssp1
data_ssp1 = pollination_table[['GRIDCODE','country',
                               'prod_poll_dep_unrealized_en_1d_ssp1','prod_poll_dep_unrealized_va_1d_ssp1','prod_poll_dep_unrealized_fo_1d_ssp1',
                               'prod_poll_dep_realized_en_1d_ssp1','prod_poll_dep_realized_va_1d_ssp1','prod_poll_dep_realized_fo_1d_ssp1',
                               'prod_total_realized_en_1d_ssp1','prod_total_realized_va_1d_ssp1','prod_total_realized_fo_1d_ssp1',
                               'nut_req_en_1d_ssp1','nut_req_va_1d_ssp1','nut_req_fo_1d_ssp1']]

data_ssp1.columns = ['GRIDCODE','name','X1_en','X1_va','X1_fo','X2_en','X2_va','X2_fo','X2+Y_en','X2+Y_va','X2+Y_fo','Z_en','Z_va','Z_fo']

#  ssp3
data_ssp3 = pollination_table[['GRIDCODE','country',
                               'prod_poll_dep_unrealized_en_1d_ssp3','prod_poll_dep_unrealized_va_1d_ssp3','prod_poll_dep_unrealized_fo_1d_ssp3',
                               'prod_poll_dep_realized_en_1d_ssp3','prod_poll_dep_realized_va_1d_ssp3','prod_poll_dep_realized_fo_1d_ssp3',
                               'prod_total_realized_en_1d_ssp3','prod_total_realized_va_1d_ssp3','prod_total_realized_fo_1d_ssp3',
                               'nut_req_en_1d_ssp3','nut_req_va_1d_ssp3','nut_req_fo_1d_ssp3']]

data_ssp3.columns = ['GRIDCODE','name','X1_en','X1_va','X1_fo','X2_en','X2_va','X2_fo','X2+Y_en','X2+Y_va','X2+Y_fo','Z_en','Z_va','Z_fo']

#  ssp5
data_ssp5 = pollination_table[['GRIDCODE','country',
                               'prod_poll_dep_unrealized_en_1d_ssp5','prod_poll_dep_unrealized_va_1d_ssp5','prod_poll_dep_unrealized_fo_1d_ssp5',
                               'prod_poll_dep_realized_en_1d_ssp5','prod_poll_dep_realized_va_1d_ssp5','prod_poll_dep_realized_fo_1d_ssp5',
                               'prod_total_realized_en_1d_ssp5','prod_total_realized_va_1d_ssp5','prod_total_realized_fo_1d_ssp5',
                               'nut_req_en_1d_ssp5','nut_req_va_1d_ssp5','nut_req_fo_1d_ssp5']]

data_ssp5.columns = ['GRIDCODE','name','X1_en','X1_va','X1_fo','X2_en','X2_va','X2_fo','X2+Y_en','X2+Y_va','X2+Y_fo','Z_en','Z_va','Z_fo']





In [9]:
def add_iso_columns (dataFrame,country_iso):
    dataFrame = pd.merge(dataFrame, country_iso,left_on='name',right_on='English short name lower case', how='outer').head(dataFrame.shape[0])
    dataFrame= dataFrame.drop(columns=['English short name lower case', 'Alpha-2 code','Numeric code','ISO 3166-2','Unnamed: 0'])
    dataFrame = dataFrame.rename(index=str, columns={"Alpha-3 code": "iso3"})

    cols = dataFrame.columns.tolist()
    cols = [cols[0]] + [cols[-1]] + cols[1:-1]
    dataFrame = dataFrame[cols]
    return dataFrame

def add_iso (dataFrame,country_iso, name):
    dataFrame = dataFrame.groupby(['name']).sum()
    dataFrame.reset_index(level=0, inplace=True)
    dataFrame= add_iso_columns(dataFrame,country_iso)
    return dataFrame
    
def create_csv(dataFrame,country_iso, name):
    dataFrame = add_iso(dataFrame,country_iso, name)
    dataFrame.to_csv(name +'.csv', sep=',')
    
def create_all_csv(data_folder):
    create_csv (data_1850,country_iso, data_folder+'1850')
    create_csv (data_1900,country_iso, data_folder+'1900')
    create_csv (data_1910,country_iso, data_folder+'1910')
    create_csv (data_1945,country_iso, data_folder+'1945')
    create_csv (data_1980,country_iso, data_folder+'1980')
    create_csv (data_cur,country_iso, data_folder+'current')
    create_csv (data_ssp1,country_iso, data_folder+'ssp1')
    create_csv (data_ssp3,country_iso, data_folder+'ssp3')
    create_csv (data_ssp5,country_iso, data_folder+'ssp5')
    


In [10]:
country_iso = pd.read_csv("country_iso.csv")
data_folder = 'data_iso/'
create_all_csv(data_folder)

In [11]:
# X2+Y folate for all the periods
mock_data = pd.DataFrame(index=data_1850.index)
mock_data = pd.concat([data_1850.name,data_1850['X2+Y_fo'],
                      data_1900['X2+Y_fo'],data_1910['X2+Y_fo'],
                      data_1945['X2+Y_fo'],data_1980['X2+Y_fo'],
                      data_cur['X2+Y_fo'],data_ssp1['X2+Y_fo'],
                      data_ssp3['X2+Y_fo'],data_ssp5['X2+Y_fo'],], axis=1)
mock_data.columns = ['name', '1850','1900', '1910','1945', '1980','cur', 'ssp1','ssp3', 'ssp5']
mock_data = add_iso (mock_data,country_iso, data_folder+'mock_data')
mock_data.to_csv('mock_data' +'.csv', sep=',')

world_data = pd.DataFrame(mock_data.sum()).transpose().drop(columns=['name','iso3'])
world_data.to_csv('world_data' +'.csv', sep=',', float_format='{:.2E}'.format)
mock_data

,name,iso3,1850,1900,1910,1945,1980,cur,ssp1,ssp3,ssp5
0,Afghanistan,AFG,1.236468e+11,1.586707e+11,1.832145e+11,3.139505e+11,4.968600e+11,5.024030e+11,4.792517e+11,6.204873e+11,6.260111e+11
1,Albania,ALB,1.505914e+11,1.731524e+11,2.505859e+11,2.563288e+11,2.577094e+11,1.754912e+11,1.757832e+11,1.757609e+11,1.752830e+11
2,Algeria,DZA,5.259982e+11,7.695124e+11,8.346817e+11,1.094752e+12,1.491157e+12,1.659518e+12,2.010545e+12,1.814659e+12,1.367727e+12
3,Angola,AGO,2.986943e+08,4.971273e+08,5.946132e+08,3.203101e+09,2.114404e+10,4.956252e+10,6.202870e+10,2.480213e+11,2.484158e+11
4,Argentina,ARG,8.478537e+12,1.065585e+13,1.235656e+13,1.643066e+13,1.735645e+13,2.289065e+13,2.311526e+13,2.401469e+13,2.418746e+13
5,Armenia,ARM,3.862154e+10,9.834729e+10,1.151005e+11,1.400173e+11,2.887856e+11,1.943986e+11,1.886737e+11,3.143613e+11,1.495827e+11
6,Aruba,ABW,-2.000000e+00,-2.000000e+00,-2.000000e+00,1.205826e+04,4.420193e+04,5.162833e+04,6.670283e+04,2.467777e+05,1.055182e+05
7,Australia,AUS,1.907309e+11,1.968503e+12,2.881833e+12,7.365846e+12,1.178608e+13,1.191393e+13,1.216691e+13,1.270666e+13,1.186855e+13
8,Austria,AUT,1.637508e+12,1.942777e+12,1.951719e+12,1.954109e+12,1.805430e+12,1.787427e+12,1.621701e+12,1.632857e+12,1.785299e+12
9,Azerbaijan,AZE,4.644285e+10,9.530189e+10,1.074959e+11,1.074960e+11,1.450363e+11,1.074966e+11,1.074937e+11,1.074937e+11,1.074933e+11


In [12]:
world_data

,1850,1900,1910,1945,1980,cur,ssp1,ssp3,ssp5
0,3.80628e+14,4.90065e+14,5.1527e+14,5.93474e+14,6.48656e+14,6.69841e+14,6.48544e+14,6.81006e+14,7.02052e+14


Questions:

1. What does the GRIDCODE actually mean? and if they are the regions how can we relate them to latitude and longitude?
2. SSP1, SSP3, SSP5 are they the future scenario description? but how are these scenarios described?
3. Difference between semi natural and natural habitats?
4. 10s – globio, 30s – gpw, monfreda 2008 – 5m, ssp – 1/8d) clearing scales?
5. What does `c_` prefix in front of columns mean?

Observations:

- GRIDCODE is unique.

In [13]:
latlong = pd.read_csv("latlong_pollination.csv")
latlong.columns = ['fid', 'long','lat']
latlong.tail()

,fid,long,lat
64795,64795,-89.5,175.5
64796,64796,-89.5,176.5
64797,64797,-89.5,177.5
64798,64798,-89.5,178.5
64799,64799,-89.5,179.5


In [14]:
pollination_table = all_datasets['pollination_table_full'].merge(latlong, left_on='fid', right_on='fid', how='inner')

In [15]:
list(pollination_table.columns)

['fid',
 'geom',
 'GRIDCODE',
 'country',
 'region',
 'PCTU5',
 'UW',
 'prod_poll_dep_realized_en_1d_ssp5',
 'nat_cont_poll_en_1d_ssp5',
 'prod_poll_dep_unrealized_en_1d_ssp5',
 'prod_total_realized_en_1d_ssp5',
 'prod_poll_dep_realized_va_1d_ssp5',
 'nat_cont_poll_va_1d_ssp5',
 'prod_poll_dep_unrealized_va_1d_ssp5',
 'prod_total_realized_va_1d_ssp5',
 'prod_poll_dep_realized_fo_1d_ssp5',
 'nat_cont_poll_fo_1d_ssp5',
 'prod_poll_dep_unrealized_fo_1d_ssp5',
 'prod_total_realized_fo_1d_ssp5',
 'poll_cont_prod_avg_1d_ssp5',
 'prod_poll_dep_realized_en_1d_ssp1',
 'nat_cont_poll_en_1d_ssp1',
 'prod_poll_dep_unrealized_en_1d_ssp1',
 'prod_total_realized_en_1d_ssp1',
 'prod_poll_dep_realized_va_1d_ssp1',
 'nat_cont_poll_va_1d_ssp1',
 'prod_poll_dep_unrealized_va_1d_ssp1',
 'prod_total_realized_va_1d_ssp1',
 'prod_poll_dep_realized_fo_1d_ssp1',
 'nat_cont_poll_fo_1d_ssp1',
 'prod_poll_dep_unrealized_fo_1d_ssp1',
 'prod_total_realized_fo_1d_ssp1',
 'poll_cont_prod_avg_1d_ssp1',
 'prod_poll_dep_

In [16]:
3.492576e+06/(3.492576e+06+2.998582e+05)

0.9209325240237523

In [17]:
pollination_table[['country','prod_poll_dep_realized_en_1d_ssp5','nat_cont_poll_en_1d_ssp5','prod_poll_dep_unrealized_en_1d_ssp5','prod_total_realized_en_1d_ssp5','nut_req_en_1d_ssp5','poll_cont_nut_req_en_1d_ssp5']]


,country,prod_poll_dep_realized_en_1d_ssp5,nat_cont_poll_en_1d_ssp5,prod_poll_dep_unrealized_en_1d_ssp5,prod_total_realized_en_1d_ssp5,nut_req_en_1d_ssp5,poll_cont_nut_req_en_1d_ssp5
0,Finland,3.492576e+06,0.920932,2.998582e+05,9.999023e+08,1.377366e+11,0.000025
1,Finland,2.081489e+08,0.821218,4.531472e+07,4.900479e+10,7.298204e+10,0.002852
2,Finland,1.755138e+09,0.523101,1.600121e+09,6.358904e+11,3.957595e+10,0.044349
3,Finland,4.447526e+08,0.701118,1.895948e+08,1.491891e+11,2.806864e+10,0.015845
4,Finland,1.596864e+06,0.982765,2.800423e+04,4.891585e+08,4.767014e+10,0.000033
5,Finland,9.325091e+05,0.961348,3.749262e+04,3.452437e+08,1.829189e+10,0.000051
6,Finland,3.533810e+08,0.618096,2.183440e+08,1.192238e+11,1.387475e+11,0.002547
7,Finland,2.121756e+09,0.494743,2.166849e+09,8.492156e+11,1.007079e+11,0.021068
8,Finland,1.684325e+09,0.625858,1.006902e+09,5.482701e+11,7.109584e+10,0.023691
9,Finland,1.058967e+09,0.538912,9.060419e+08,4.028124e+11,3.268732e+10,0.032397


In [18]:
unxncp = all_datasets['unxncp'].merge(latlong, left_on='fid', right_on='fid', how='inner')

In [19]:
list(unxncp.columns)

['fid',
 'GRIDCODE',
 'region',
 'country',
 'prod_poll_dep_unrealized_en_1d_cur',
 'prod_poll_dep_unrealized_fo_1d_cur',
 'prod_poll_dep_unrealized_va_1d_cur',
 'prod_poll_dep_unrealized_en_1d_ssp1',
 'prod_poll_dep_unrealized_fo_1d_ssp1',
 'prod_poll_dep_unrealized_va_1d_ssp1',
 'prod_poll_dep_unrealized_en_1d_ssp3',
 'prod_poll_dep_unrealized_fo_1d_ssp3',
 'prod_poll_dep_unrealized_va_1d_ssp3',
 'prod_poll_dep_unrealized_en_1d_ssp5',
 'prod_poll_dep_unrealized_fo_1d_ssp5',
 'prod_poll_dep_unrealized_va_1d_ssp5',
 'nat_cont_poll_en_1d_cur',
 'nat_cont_poll_fo_1d_cur',
 'nat_cont_poll_va_1d_cur',
 'nat_cont_poll_en_1d_ssp1',
 'nat_cont_poll_fo_1d_ssp1',
 'nat_cont_poll_va_1d_ssp1',
 'nat_cont_poll_en_1d_ssp3',
 'nat_cont_poll_fo_1d_ssp3',
 'nat_cont_poll_va_1d_ssp3',
 'nat_cont_poll_en_1d_ssp5',
 'nat_cont_poll_fo_1d_ssp5',
 'nat_cont_poll_va_1d_ssp5',
 'prod_poll_dep_realized_en_1d_cur',
 'prod_poll_dep_realized_en_1d_ssp1',
 'prod_poll_dep_realized_en_1d_ssp3',
 'prod_poll_dep_reali

Nature's contribution to pollination is the percentage of prod_poll_dep_realized/ the total production

In [20]:
unxncp[['prod_poll_dep_unrealized_en_1d_cur', 'nat_cont_poll_en_1d_cur', 'prod_poll_dep_realized_en_1d_cur','UN_cur','NCP_cur']]

,prod_poll_dep_unrealized_en_1d_cur,nat_cont_poll_en_1d_cur,prod_poll_dep_realized_en_1d_cur,UN_cur,NCP_cur
0,0.000000e+00,1.000000,3.792434e+06,0.000000,0.333333
1,3.076646e+07,0.878616,2.226972e+08,479.207931,0.859045
2,1.399546e+09,0.587343,1.992002e+09,23750.552670,0.522512
3,4.935109e+07,0.922920,5.909089e+08,630.529173,0.896613
4,0.000000e+00,1.000000,1.624868e+06,0.000000,1.000000
5,1.038155e+04,0.989298,9.596202e+05,0.001042,0.329766
6,1.436549e+08,0.751328,4.340336e+08,1390.588201,0.731784
7,1.486380e+09,0.655732,2.831128e+09,20726.513456,0.600815
8,5.804089e+08,0.786324,2.135897e+09,10276.934762,0.718207
9,6.833295e+08,0.654151,1.292475e+09,9584.241290,0.589205


# PERCENTAGES

### For points on the map

In [21]:
resource = 'en'
resolution = '1d'
periods = ['1850','1900','1910','1945','1980','cur','ssp1','ssp3','ssp5']

In [22]:
columns_totake = []
for period in periods :
    columns_totake.append('nat_cont_poll_'+resource+'_'+resolution+'_'+period)

columns_totransform = columns_totake.copy()
columns_totake.extend(('long', 'lat'))

natures_contribution_pixels = pollination_table['country']
natures_contribution_pixels = pd.concat([natures_contribution_pixels, pollination_table[columns_totake].copy()], axis=1)
for col in columns_totransform:
    natures_contribution_pixels[col] = (natures_contribution_pixels[col]*10000).round()/100

column_names = ['country']
for col in periods:
    column_names.append(col)

column_names.append('long')
column_names.append('lat')
for i in range(7,10):
    column_names[i] = column_names[i].upper()

column_names[6] = '2015'
natures_contribution_pixels.columns = column_names
natures_contribution_pixels.head()


,country,1850,1900,1910,1945,1980,2015,SSP1,SSP3,SSP5,long,lat
0,Finland,100.00,100.00,100.00,100.00,100.00,100.00,99.43,99.67,92.09,65.5,25.5
1,Finland,87.16,80.55,74.59,71.56,85.38,87.86,87.60,87.60,82.12,64.5,24.5
2,Finland,58.77,55.52,53.28,51.56,56.77,58.73,55.75,55.96,52.31,64.5,25.5
3,Finland,93.28,90.92,90.91,90.81,90.08,92.29,79.11,79.30,70.11,64.5,26.5
4,Finland,100.00,100.00,100.00,100.00,100.00,100.00,98.41,98.49,98.28,64.5,27.5


In [23]:
natures_contribution_pixels.head(5)

,country,1850,1900,1910,1945,1980,2015,SSP1,SSP3,SSP5,long,lat
0,Finland,100.00,100.00,100.00,100.00,100.00,100.00,99.43,99.67,92.09,65.5,25.5
1,Finland,87.16,80.55,74.59,71.56,85.38,87.86,87.60,87.60,82.12,64.5,24.5
2,Finland,58.77,55.52,53.28,51.56,56.77,58.73,55.75,55.96,52.31,64.5,25.5
3,Finland,93.28,90.92,90.91,90.81,90.08,92.29,79.11,79.30,70.11,64.5,26.5
4,Finland,100.00,100.00,100.00,100.00,100.00,100.00,98.41,98.49,98.28,64.5,27.5


In [24]:
natures_contribution_pixels.to_csv('pixel_energy.csv', sep=',')

### Find min max latitudes

In [25]:
mins = natures_contribution_pixels.groupby('country').idxmin().add_suffix('_min')

maxs = natures_contribution_pixels.groupby('country').idxmax().add_suffix('_max')
countriesdf = natures_contribution_pixels.groupby('country').count().reset_index()
mins.head()

,1850_min,1900_min,1910_min,1945_min,1980_min,2015_min,SSP1_min,SSP3_min,SSP5_min,lat_min,long_min
country,,,,,,,,,,,
Afghanistan,3221,3221,3377,3377,3534,3534,3534,3534,3534,3949,4397
Albania,2707,2707,2555,2707,2707,2707,2707,2707,2707,2554,2861
Algeria,3925,3648,3648,3648,3648,3648,3481,3648,3648,3642,3925
Angola,7294,7294,7294,7389,7489,7776,7776,7489,7996,7293,8432
Argentina,8581,8581,8581,8640,8640,8640,8640,8640,8640,9501,9508


In [26]:
natures_contribution_pixels.iloc[2707,:]

country    Albania
1850          8.27
1900          5.77
1910          6.26
1945          6.61
1980          7.64
2015           9.3
SSP1         11.92
SSP3         11.66
SSP5          9.49
long          41.5
lat           19.5
Name: 2707, dtype: object

In [27]:
fids = pollination_table[['fid','lat','long']]
natures_contribution_pixels_min_max = pd.DataFrame(countriesdf['country'])
natures_contribution_pixels_min_max

periodnames = ['1850', '1900', '1910', '1945', '1980', '2015', 'SSP1', 'SSP3', 'SSP5']

for period in periodnames:
    natures_contribution_pixels_min_max[period+'_lat_min'] = pd.DataFrame(fids.iloc[mins[period+'_min'],:].lat).reset_index().drop(columns=['index'])
    natures_contribution_pixels_min_max[period+'_long_min'] = pd.DataFrame(fids.iloc[mins[period+'_min'],:].long).reset_index().drop(columns=['index'])
    natures_contribution_pixels_min_max[period+'_lat_max'] = pd.DataFrame(fids.iloc[maxs[period+'_max'],:].lat).reset_index().drop(columns=['index'])
    natures_contribution_pixels_min_max[period+'_long_max'] = pd.DataFrame(fids.iloc[maxs[period+'_max'],:].long).reset_index().drop(columns=['index'])

natures_contribution_pixels_min_max.head()


,country,1850_lat_min,1850_long_min,1850_lat_max,1850_long_max,1900_lat_min,1900_long_min,1900_lat_max,1900_long_max,1910_lat_min,...,SSP1_lat_max,SSP1_long_max,SSP3_lat_min,SSP3_long_min,SSP3_lat_max,SSP3_long_max,SSP5_lat_min,SSP5_long_min,SSP5_lat_max,SSP5_long_max
0,Afghanistan,71.5,38.5,70.5,37.5,71.5,38.5,70.5,37.5,73.5,...,73.5,37.5,63.5,36.5,74.5,37.5,63.5,36.5,74.5,37.5
1,Albania,19.5,41.5,19.5,42.5,19.5,41.5,19.5,42.5,20.5,...,19.5,42.5,19.5,41.5,19.5,42.5,19.5,41.5,19.5,42.5
2,Algeria,3.5,33.5,2.5,34.5,3.5,35.5,5.5,33.5,3.5,...,-2.5,35.5,3.5,35.5,7.5,33.5,3.5,35.5,5.5,33.5
3,Angola,13.5,-4.5,12.5,-4.5,13.5,-4.5,12.5,-4.5,13.5,...,15.5,-6.5,15.5,-6.5,22.5,-9.5,16.5,-12.5,20.5,-12.5
4,Argentina,-65.5,-21.5,-64.5,-22.5,-65.5,-21.5,-63.5,-21.5,-65.5,...,-62.5,-21.5,-66.5,-22.5,-62.5,-22.5,-66.5,-22.5,-62.5,-22.5


In [28]:
natures_contribution_pixels_min_max.to_csv('min_max_coordinates_by_country.csv',sep=',')

### For countries on the map

In [29]:
natures_contribution_countries = pollination_table[['country']].copy()
natures_contribution_countries.columns= ['name']

for period in periods:
        natures_contribution_countries['realized'+period] = pollination_table['prod_poll_dep_realized_en_1d_'+period]
        natures_contribution_countries['unrealized'+period] = pollination_table['prod_poll_dep_unrealized_en_1d_'+period]

#     natures_contribution_countries[period] = (pollination_table['prod_poll_dep_realized_en_1d_'+period]/(pollination_table['prod_poll_dep_realized_en_1d_'+period] +pollination_table['prod_poll_dep_unrealized_en_1d_'+period])*10000).round()/100
natures_contribution_countries.sample(5)


,name,realized1850,unrealized1850,realized1900,unrealized1900,realized1910,unrealized1910,realized1945,unrealized1945,realized1980,unrealized1980,realizedcur,unrealizedcur,realizedssp1,unrealizedssp1,realizedssp3,unrealizedssp3,realizedssp5,unrealizedssp5
2728,Georgia,2.398644e+07,7.851582e+07,5.106351e+07,6.890339e+08,6.779245e+07,1.164328e+09,4.323873e+08,2.168151e+09,6.931762e+08,8.578744e+09,9.594257e+07,2.504596e+09,2.959206e+08,2.301285e+09,3.490763e+08,6.553736e+09,9.332730e+07,2.503878e+09
2393,Croatia,2.739819e+09,7.870665e+07,1.246089e+10,6.793356e+08,1.260009e+10,6.844004e+08,1.279358e+10,6.850433e+08,1.279818e+10,6.804383e+08,4.227086e+09,1.411384e+08,4.227086e+09,1.411384e+08,4.227086e+09,1.411384e+08,4.226833e+09,1.413916e+08
4494,Iran (Islamic Republic of),-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,3.011625e+04,3.990393e+06,-1.000000e+00,-1.000000e+00
5759,Burkina Faso,6.941002e+07,6.948673e+09,7.844003e+07,8.223553e+09,8.133840e+07,8.682428e+09,1.068636e+08,1.425421e+10,1.321340e+08,2.381243e+10,1.484571e+08,3.491167e+10,1.484571e+08,3.477928e+10,1.527372e+08,3.775864e+10,1.527372e+08,3.774174e+10
6496,Colombia,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,3.849809e-11,5.121295e+06,4.131528e+06,1.731002e+08,4.131528e+06,1.896401e+08,4.131528e+06,1.896401e+08,2.338500e+07,8.513244e+08,5.435396e+06,2.736830e+08


In [30]:
natures_contribution_countries =add_iso (natures_contribution_countries,country_iso, 'name')

In [31]:
final_natures_cont_countries = natures_contribution_countries[['name','iso3']].copy()
for period in periods:
    final_natures_cont_countries[period] = ((natures_contribution_countries['realized'+period]/(natures_contribution_countries['realized'+period] +natures_contribution_countries['unrealized'+period]))*10000).fillna(50).astype(int)/100.


column_names = final_natures_cont_countries.columns.tolist()

for i in range(8,11):
    column_names[i] = column_names[i].upper()
    
column_names[7] = '2015'
final_natures_cont_countries.columns = column_names

columns_tochange = ['1850','1900','1910','1945','1980','2015','SSP1','SSP3','SSP5']
for col in columns_tochange:
    final_natures_cont_countries[col][final_natures_cont_countries[col] < 0] = 0
final_natures_cont_countries.head()



/anaconda3/envs/dataviz/lib/python3.7/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,name,iso3,1850,1900,1910,1945,1980,2015,SSP1,SSP3,SSP5
0,Afghanistan,AFG,7.67,7.48,8.16,7.70,4.00,3.79,5.10,3.17,3.86
1,Albania,ALB,46.71,13.35,12.45,16.28,29.17,48.45,52.56,52.25,45.29
2,Algeria,DZA,20.62,8.44,6.72,3.64,1.92,1.31,0.67,0.75,1.14
3,Angola,AGO,57.73,72.24,75.91,82.82,86.34,80.87,76.41,26.52,52.63
4,Argentina,ARG,45.93,30.83,22.39,7.24,5.14,3.47,4.39,2.81,2.94


In [32]:
final_natures_cont_countries.to_csv('country_energy.csv', sep=',')

In [33]:
final_natures_cont_countries.iloc[99,:]

name    Mongolia
iso3         MNG
1850           0
1900       49.16
1910       48.98
1945       25.19
1980           0
2015           0
SSP1           0
SSP3           0
SSP5         8.7
Name: 99, dtype: object